In [13]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

df_2015 = pd.read_csv("2015.csv")
df_2016 = pd.read_csv("2016.csv")
df_2017 = pd.read_csv("2017.csv")


def standardize_columns(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(
        ' ', '_').str.replace('(', '').str.replace(')', '').str.replace('..', '_')
    return df


df_2015 = standardize_columns(df_2015)
df_2016 = standardize_columns(df_2016)
df_2017 = standardize_columns(df_2017)

df_2015['year'] = 2015
df_2016['year'] = 2016
df_2017['year'] = 2017

full_df = pd.concat([df_2015, df_2016, df_2017], ignore_index=True)
countries = full_df['country'].unique()

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Аналіз рівня щастя", style={'textAlign': 'center'}),

    html.Div([
        dcc.Dropdown(
            id='country-dropdown',
            options=[{'label': country, 'value': country}
                     for country in countries],
            value='Switzerland',
            style={'width': '50%'}
        ),
    ], style={'padding': '20px', 'textAlign': 'center'}),

    html.H2("Загальний огляд", style={'textAlign': 'center'}),
    html.Div([
        dcc.Graph(id='overall-graph1'),
        dcc.Graph(id='overall-graph2'),
        dcc.Graph(id='happiness-graph9'),
        dcc.Graph(id='overall-graph3'),
        dcc.Graph(id='overall-graph4'),
        dcc.Graph(id='happiness-graph10'),
    ], style={'display': 'grid', 'gridTemplateColumns': 'repeat(3, 1fr)', 'gap': '20px', 'padding': '20px'}),

    html.H2("Графіки по обраній країні", style={'textAlign': 'center'}),
    html.Div([
        dcc.Graph(id='happiness-graph1'),
        dcc.Graph(id='happiness-graph2'),
        dcc.Graph(id='happiness-graph3'),
        dcc.Graph(id='happiness-graph4'),
        dcc.Graph(id='happiness-graph5'),
        dcc.Graph(id='happiness-graph6'),
        dcc.Graph(id='happiness-graph7'),
        dcc.Graph(id='happiness-graph8'),
    ], style={'display': 'grid', 'gridTemplateColumns': 'repeat(3, 1fr)', 'gap': '20px', 'padding': '20px'}),
])


@app.callback(
    [Output('overall-graph1', 'figure'),
     Output('overall-graph2', 'figure'),
     Output('overall-graph3', 'figure'),
     Output('overall-graph4', 'figure'),
     Output('happiness-graph1', 'figure'),
     Output('happiness-graph2', 'figure'),
     Output('happiness-graph3', 'figure'),
     Output('happiness-graph4', 'figure'),
     Output('happiness-graph5', 'figure'),
     Output('happiness-graph6', 'figure'),
     Output('happiness-graph8', 'figure'),
     Output('happiness-graph9', 'figure'),
     Output('happiness-graph10', 'figure')],
    [Input('country-dropdown', 'value')]
)
def update_graphs(selected_country):
    country_data = full_df[full_df['country'] == selected_country]
    latest_data = country_data[country_data['year']
                               == country_data['year'].max()].iloc[0]

    avg_score_year = full_df.groupby(
        'year')['happiness_score'].mean().reset_index()
    fig_o1 = px.line(avg_score_year, x='year', y='happiness_score',
                     title='Середній рівень щастя по роках')

    avg_factors = full_df.groupby(
        'year')[['economy', 'trustgovernmentcorruption', 'health', 'freedom']].mean().reset_index()
    fig_o2 = px.line(avg_factors, x='year', y=['economy', 'trustgovernmentcorruption', 'health', 'freedom'],
                     title='Середні значення основних факторів по роках')

    top_10_2017 = df_2017.sort_values(
        'happiness_score', ascending=False).head(10)
    fig_o3 = px.bar(top_10_2017, x='country', y='happiness_score',
                    title='Топ-10 країн за щастям (2017)')

    region_avg = full_df.groupby('region')['happiness_score'].mean(
    ).sort_values(ascending=False).reset_index()
    fig_o4 = px.bar(region_avg, x='region', y='happiness_score',
                    title='Середній рівень щастя по регіонах')

    fig1 = px.line(country_data, x='year', y='happiness_score',
                   title=f'Динаміка рівня щастя в {selected_country}')
    fig2 = px.line(country_data, x='year', y='happiness_rank',
                   title=f'Позиція {selected_country} в рейтингу')
    region_data = full_df[full_df['region'] == country_data['region'].iloc[0]]
    fig3 = px.line(region_data, x='year', y='happiness_score', color='country',
                   title=f'{selected_country} vs інші країни в регіоні')
    fig4 = px.bar(x=['Економіка', 'Сім’я', 'Здоров’я', 'Свобода', 'Довіра до уряду', 'Щедрість'],
                  y=[latest_data['economy'], latest_data['family'], latest_data['health'],
                     latest_data['freedom'], latest_data['trustgovernmentcorruption'], latest_data['generosity']],
                  title=f'Фактори щастя в {selected_country} ({latest_data["year"]})')
    fig5 = px.line(country_data, x='year',
                   y=['economy', 'trustgovernmentcorruption', 'health', 'freedom'],
                   title=f'Ключові фактори щастя по роках в {selected_country}')

    world_avg_2017 = df_2017[['economy', 'family', 'health',
                              'freedom', 'trustgovernmentcorruption', 'generosity']].mean()
    comparison = pd.DataFrame({
        'Фактори': ['Економіка', 'Сім’я', 'Здоров’я', 'Свобода', 'Довіра', 'Щедрість'],
        'Країна': [latest_data['economy'], latest_data['family'], latest_data['health'],
                   latest_data['freedom'], latest_data['trustgovernmentcorruption'], latest_data['generosity']],
        'Світовий середній': world_avg_2017.values
    })
    fig6 = px.bar(comparison, x='Фактори', y=['Країна', 'Світовий середній'],
                  barmode='group', title=f'{selected_country} vs світовий середній (2017)')

    fig8 = px.bar(country_data, x='year', y='happiness_rank',
                  title=f'Місце в рейтингу по роках ({selected_country})')

    rank_2015 = df_2015[['country', 'happiness_rank']].rename(
        columns={'happiness_rank': 'rank_2015'})
    rank_2017 = df_2017[['country', 'happiness_rank']].rename(
        columns={'happiness_rank': 'rank_2017'})
    df_rank_change = pd.merge(rank_2015, rank_2017, on='country')
    df_rank_change['rank_change'] = df_rank_change['rank_2015'] - \
        df_rank_change['rank_2017']

    fig9 = px.bar(df_rank_change.sort_values('rank_change', ascending=False).head(20),
                  x='country', y='rank_change',
                  title='Топ 20 країн з покращенням рейтингу (2015–2017)',
                  labels={'rank_change': 'Зміна (↑ — покращення)'})
    fig10 = px.bar(df_rank_change.sort_values('rank_change').head(20),
                   x='country', y='rank_change',
                   title='Топ 20 країн з погіршенням рейтингу (2015–2017)',
                   labels={'rank_change': 'Зміна (↓ — погіршення)'},
                   )

    return fig_o1, fig_o2, fig_o3, fig_o4, fig1, fig2, fig3, fig4, fig5, fig6,  fig8, fig9, fig10


if __name__ == '__main__':
    app.run(debug=True)